<a href="https://www.kaggle.com/code/nerminmostafa/vgg16-mnist?scriptVersionId=160185481" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import seaborn as sns
import tensorflow as tf
from keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras import utils
from keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import datasets, layers, models, Sequential
from keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, ImageDataGenerator
from keras.layers import Dense, Activation, Dropout, Input, Conv2D, MaxPooling2D, Flatten, ReLU, AveragePooling2D, add

# MNIST Dataset

In [ ]:
(mnist_xTrain, mnist_yTrain), (mnist_xTest, mnist_yTest) = datasets.mnist.load_data()

In [ ]:
print('X Train Shape: ', mnist_xTrain.shape)
print('X Test Shape:  ', mnist_xTest.shape)
print('\nY Train Shape: ', mnist_yTrain.shape)
print('Y Test Shape:  ', mnist_yTest.shape)

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=5, figsize=(5, 5))
axs = axs.flatten()

for i in range(20):
    img = mnist_xTrain[i]
    axs[i].imshow(img, cmap='gray')
    axs[i].set_title("Label: {}".format(mnist_yTrain[i]))

plt.tight_layout()
plt.show()

In [ ]:
mnist_yTrain = utils.to_categorical(mnist_yTrain)
mnist_yTest = utils.to_categorical(mnist_yTest)

## VGG-16 Architecutre on MNIST Dataset

In [ ]:
from keras.layers import GlobalAveragePooling2D

input_layer = Input(shape=(28, 28, 1))

# Block 1
x = Conv2D(64, (3, 3), padding='same', activation='relu', name='block1_conv1')(input_layer)
x = Conv2D(64, (3, 3), padding='same', activation='relu', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), padding='same', activation='relu', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), padding='same', activation='relu', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv3')(x)
# x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# Global Average Pooling
x = GlobalAveragePooling2D()(x)

# Fully Connected 1
x = Dense(256, activation='relu')(x)

# Fully Connected 2
x = Dense(256, activation='relu')(x)

# Output Layer (SoftMax)
outputLayer = Dense(10, activation='softmax')(x)

vgg_mnist = Model(inputs=input_layer, outputs=outputLayer)

- Compile the model

In [ ]:
vgg_mnist.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vgg_mnist.summary()
plot_model(vgg_mnist, show_shapes=True, to_file='vgg_model_mnist.png')

- Train Model

In [ ]:
vgg_mnist_history = vgg_mnist.fit(mnist_xTrain, mnist_yTrain, validation_data=(mnist_xTest, mnist_yTest), epochs=10)

- Make Predictions

In [ ]:
vgg_mnist_perdict = vgg_mnist.predict(mnist_xTest)

vgg_mnist_perdict = [np.argmax(arr) for arr in vgg_mnist_perdict]
print(np.array(vgg_mnist_perdict))

In [ ]:
mnist_ytest = [np.argmax(arr) for arr in mnist_yTest]
print(np.array(mnist_ytest))

- Evaluate Model

In [ ]:
vgg_mnist.evaluate(mnist_xTest,mnist_yTest)

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(vgg_mnist_history.history['accuracy'])
plt.plot(vgg_mnist_history.history['val_accuracy'])
plt.title('VGG-16 Architecture Accuracy for MNIST Dataset')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(vgg_mnist_history.history['loss'])
plt.plot(vgg_mnist_history.history['val_loss'])
plt.title('VGG-16 Architecture Loss for MNIST Dataset')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.show()

In [ ]:
print('Accuracy Score =',round(np.max(vgg_mnist_history.history['val_accuracy']) * 100, 2) , '%')

## Pretrained VGG-16 on MNIST Dataset

In [ ]:
x_train = np.dstack([mnist_xTrain] * 3)
x_test = np.dstack([mnist_xTest] * 3)

x_train = x_train.reshape(-1, 28, 28, 3)
x_test = x_test.reshape(-1, 28, 28, 3)

In [ ]:
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

In [ ]:
pre_vgg_model = models.Sequential()
pre_vgg_model.add(base_model)

# Flatten and add dense layers for MNIST classification
pre_vgg_model.add(layers.Flatten())
pre_vgg_model.add(layers.Dense(256, activation='relu'))
pre_vgg_model.add(layers.Dropout(0.5))
pre_vgg_model.add(layers.Dense(10, activation='softmax'))

x_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_train])
x_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_test])

pre_vgg_model.summary()

In [ ]:
pre_vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
pre_vgg_history = pre_vgg_model.fit(x_train, mnist_yTrain, epochs=10, validation_data=(x_test, mnist_yTest))

In [ ]:
pretrainedVggMnistPerdict = pre_vgg_model.predict(x_test)

pretrainedVggMnistPerdict = [np.argmax(arr) for arr in pretrainedVggMnistPerdict]
print(np.array(pretrainedVggMnistPerdict))

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(pre_vgg_history.history['accuracy'])
plt.plot(pre_vgg_history.history['val_accuracy'])
plt.title('PreTrained VGG-16 Architecture Accuracy for MNIST Dataset')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(pre_vgg_history.history['loss'])
plt.plot(pre_vgg_history.history['val_loss'])
plt.title('PreTrained VGG-16 Architecture Loss for MNIST Dataset')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.show()

In [ ]:
print('Accuracy Score =',round(np.max(pre_vgg_history.history['val_accuracy']) * 100, 2) , '%')